Model machine learning dan deep learning memerlukan semua variabel input dan output dalam bentuk numerik. Ini berarti bahwa jika data Anda mengandung data kategorikal, Anda harus mengodekannya menjadi angka sebelum Anda dapat menyesuaikan dan mengevaluasi model. Ada beberapa teknik untuk mengodekan variabel kategorikal. Mari kita bahas ini:

1. **One-Hot Encoding**
2. **Label Encoding**
3. **Target Encoding**
4. **Target Guided Ordinal Encoding**
5. **Custom Encoding**

# Data Preparation

In [92]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder, TargetEncoder
from sklearn.model_selection import train_test_split

In [44]:
data = pd.read_csv('train.csv')
data = data[['Sex','Cabin','Embarked','Survived']]
data.head(3)

,Sex,Cabin,Embarked,Survived
0,male,NaN,S,0
1,female,C85,C,1
2,female,NaN,S,1


In [45]:
#checking missing values
data.isnull().sum()

Sex           0
Cabin       687
Embarked      2
Survived      0
dtype: int64

In [46]:
#non imputed data
df_nn = data.copy()
df = data.drop(columns=['Survived'])
y = data['Survived']

# We will fill the missing values
imp = SimpleImputer(strategy="most_frequent")
imp.fit(df)
df[df.columns] = imp.transform(df)


In [47]:
# train test split

X_train,X_test,y_train,y_test = train_test_split(df,y)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(668, 3)
(223, 3)
(668,)
(223,)


# 1. Onehot Encoding

## Menggunakan sklearn Onehot Encoder

Langkah pertama adalah kita akan meng-copy dari train test split ke dataframe baru, sehingga hasil dari data pre-processing tidak akan mengganggu data orisinal

In [48]:
df_train = X_train.copy()
df_test = X_test.copy()

In [49]:
df_train_category = df_train[['Sex', 'Embarked']]
df_test_category = df_test[['Sex', 'Embarked']]

In [50]:
one = OneHotEncoder(handle_unknown ='ignore')
one.fit(df_train_category)
train_vec = one.transform(df_train_category)
test_vec = one.transform(df_test_category)

In [51]:
one.get_feature_names_out()

array(['Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype=object)

**Catatan:**

1. `handle_unknown = 'ignore'` sangat penting. Jika kategori yang tidak dikenal ditemukan selama transformasi, kolom yang dihasilkan untuk one-hot encoding dari fitur ini akan semuanya nol. Dalam kasus sebelumnya, kita menyelaraskan dan mengisi kategori yang tidak dikenal dari data uji dengan nol. Di sini, ini akan ditangani secara otomatis.

2. Hal penting lainnya adalah bahwa secara default, one hot encoder akan mengembalikan vektor jarang (sparse). Ini akan berguna untuk perhitungan yang lebih cepat saat menangani data berdimensi tinggi. Kita juga bisa mematikannya dengan mengatur `sparse = False` jika kita ingin mengonversinya menjadi padat (dense) jika kita ingin melihat data frame.

In [52]:
feats = one.get_feature_names_out()
# sparce to dense
train_vec = train_vec.toarray()
test_vec = test_vec.toarray()


train_df_category = pd.DataFrame(train_vec,columns=feats)
test_df_category = pd.DataFrame(test_vec,columns=feats)

print(train_df_category.shape)
print(test_df_category.shape)

(668, 5)
(223, 5)


In [53]:
train_df_category.head(3)

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.0,1.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,1.0


In [54]:
test_df_category.head(3)

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.0,1.0,1.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0


**Catatan:**

Contoh untuk mendapatkan vektor non-sparse (padat) secara langsung.

Tidak terdapat perbedaan pada hasil output, hanya cara penyimpanan nya yang berbeda, sehingga lebih hemat memori. Ini sangat disarankan pada kasus categorical yang sangat variatif. Contoh pada kasus warna baju T-Shirt, yang terdapat pilihan warna bisa lebih dari 10.

Contoh ilustrasi:
```
["merah", "biru", "hijau", "merah"]

[[1, 0, 0],  # merah
 [0, 0, 1],  # biru
 [0, 1, 0],  # hijau
 [1, 0, 0]]  # merah
```

Cara ubah ke sparse adalah mengubah hasil encoding ke format ---> **(index_data, posisi_angka_1_dalam_array)**
```
(0, 0)  1
(1, 2)  1
(2, 1)  1
(3, 0)  1
```

In [82]:
dff = df[['Sex', 'Embarked']].copy()
one = OneHotEncoder(sparse_output=True)
one.fit(dff)
dff_ = one.transform(dff)
feats = one.get_feature_names_out()
dff_ = pd.DataFrame(dff_.toarray(),columns=feats)
print(dff_.shape)

(891, 5)


In [83]:
dff_.head()

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0


In [84]:
dff = df[['Sex', 'Embarked']].copy()
one = OneHotEncoder(sparse_output=False)
one.fit(dff)
dff_ = one.transform(dff)
feats = one.get_feature_names_out()
dff_ = pd.DataFrame(dff_,columns=feats)
print(dff_.shape)

(891, 5)


In [85]:
dff_.head()

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0


**Catatan:** 

Menghapus fitur one-hot pertama dari setiap kolom. Ini akan sangat membantu terutama dalam kasus model linear di mana multikolinearitas dapat mempengaruhi model.

***INGAT!!!***
---> Penanganan dummy variable pada one hot encoding : (n-1) kolom

Dengan menambahkan ```drop='first'```, maka kolom pertama dari hasil encoding dari setiap variabel categorical akan dihapus

In [86]:
dff = df[['Sex', 'Embarked']].copy()
one = OneHotEncoder(drop='first',sparse_output=False)
one.fit(dff)
dff_ = one.transform(dff)
feats = one.get_feature_names_out()
dff_ = pd.DataFrame(dff_,columns=feats)
print(dff_.shape)


(891, 3)


In [87]:
dff_.head()

,Sex_male,Embarked_Q,Embarked_S
0,1.0,0.0,1.0
1,0.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,1.0,0.0,1.0


# 2. Label Encoding

**Catatan:** Dalam label encoding, setiap label diubah menjadi nilai integer. Kita akan membuat variabel yang berisi kategori yang mewakili kualifikasi pendidikan seseorang.

### (a) Menggunakan pendekatan kode kategori

Pendekatan ini memerlukan kolom kategori untuk memiliki tipe data ``'category'``. Secara default, kolom non-numerik adalah tipe ``'object'``. Jadi, Anda mungkin perlu mengubah tipe menjadi `'category'` sebelum menggunakan pendekatan ini.

In [90]:
dff = df[['Sex', 'Embarked']].copy()
for feat in dff.columns:
    dff[feat] = dff[feat].astype('category')
    dff[feat] = dff[feat].cat.codes
dff.head()

,Sex,Embarked
0,1,2
1,0,0
2,0,2
3,0,2
4,1,2


### (b) using Sklearn Label Encoder

In [91]:
dff = df[['Sex', 'Embarked']].copy()
for feat in dff.columns:
    lb = LabelEncoder()
    lb.fit(dff[feat])
    dff[feat] = lb.transform(dff[feat])

print(dff.shape)
dff.head()

(891, 2)


,Sex,Embarked
0,1,2
1,0,0
2,0,2
3,0,2
4,1,2


# 3. Target Encoding

Berikut adalah teknik encoding yang merupakan teknik untuk mengubah nilai kategorial dengan nilai rata-rata variabel target yang berkaitan dengan variabel kategorial itu sendiri.

Target Encoding berbeda dengan teknik encoding lainnya, yakni memerlukan nilai `y` atau `target`. 

Oleh sebab itu, encoding ini hanya cocok pada kasus **Supervised Learning**

In [98]:
dff = df[['Sex', 'Embarked']].copy()
for feat in dff.columns:
    tg = TargetEncoder()
    tg.fit(dff[[feat]], y)
    dff[feat] = tg.transform(dff[[feat]])

print(dff.shape)
dff.head()

(891, 2)


,Sex,Embarked
0,0.189127,0.339075
1,0.741117,0.552522
2,0.741117,0.339075
3,0.741117,0.339075
4,0.189127,0.339075


# 4. Target Guided Ordinal Encoding

Sama dengann Target Encoding, hanya terdapat tambahan, yakni sistem ranking (ordinal encoding). Dimana dari hasil rata-rata, diurutkan dengan nilai tertinggi, kemudian dituliskan rankingnya

In [106]:
dff = df[['Sex', 'Embarked']].copy()
for feat in dff.columns:
    ## Target Encoding
    tg = TargetEncoder()
    tg.fit(dff[[feat]], y)
    dff[feat] = tg.transform(dff[[feat]])

    ## Below the ranking process part
    set_mean_list = list(set(dff[feat].values))
    set_mean_list.sort()
    transformation_dict = {value: key for key, value in enumerate(set_mean_list)}
    dff[feat] = dff[feat].map(transformation_dict)

print(dff.shape)
dff.head()

(891, 2)


,Sex,Embarked
0,0,0
1,1,2
2,1,0
3,1,0
4,0,0


# 5. Custom Encoding

Di sini, kita hanya mengubah kategori menjadi angka yang kita inginkan. Tentu kita harus memiliki *rules* atau aturan yang tetap untuk melakukan ini. Selain itu, juga ada basis alasan yang kuat akan penggunaan *rules* tersebut.

In [22]:
df_train = X_train.copy()
df_test = X_test.copy()

In [23]:
df['Cabin'].value_counts()

B96 B98        691
G6               4
C23 C25 C27      4
D                3
C22 C26          3
              ... 
F E69            1
C103             1
C46              1
C49              1
D6               1
Name: Cabin, Length: 147, dtype: int64

Dalam kasus Cabin, kita dapat melihat bahwa ada 147 nilai unik. Namun, dari nilai-nilai ini, cabin 'B96 B98' muncul sebanyak 691 kali, sementara yang lainnya muncul hanya dalam jumlah yang sangat sedikit. Jadi, misalkan kita ingin mengkodekan 'B96 B98' sebagai 1 dan sisanya sebagai 0. Jenis encoding ini disebut sebagai custom encoding. Mari kita lihat bagaimana kita dapat melakukannya.

In [108]:

df_train['Cabin_encoded'] = df_train['Cabin'].apply(lambda x: 1 if x=='B96 B98' else 0)
df_test['Cabin_encoded'] = df_test['Cabin'].apply(lambda x: 1 if x=='B96 B98' else 0)
df_train['Cabin_encoded']

405    1
639    1
168    1
230    0
715    0
      ..
135    1
836    1
839    0
476    1
111    1
Name: Cabin_encoded, Length: 668, dtype: int64